# 创建文件并写入数据
- os.makedirs(os.path.join('..', 'data'), exist_ok=True)  
    exist_ok = True 如果文件存在久不会抛出异常
    
os.path.join(path, *paths)
合理地拼接一个或多个路径部分。返回值是 path 和 *paths 所有值的连接，每个非空部分后面都紧跟一个目录分隔符 (os.sep)，除了最后一部分。这意味着如果最后一部分为空，则结果将以分隔符结尾。如果参数中某个部分是绝对路径，则绝对路径前的路径都将被丢弃，并从绝对路径部分开始连接。

In [11]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

读取数据

In [12]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [13]:
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


**处理缺省值**

In [14]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs # 取出前两列

,NumRooms,Alley
0,NaN,Pave
1,2.0,NaN
2,4.0,NaN
3,NaN,NaN


In [15]:
outputs # 取出第三列

0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64

对于NaN 使用同一列的均值，进行填充

In [16]:
inputs = inputs.fillna(inputs.mean())
inputs # 如果只选择 有效列，就不会抛出 FutureWarning

/var/folders/r2/gdmq3jbj7fjcm4vwnxszxy9m0000gp/T/ipykernel_2204/1283016260.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。**离散成两列**

In [17]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


# 转换张量格式

In [20]:
import torch
# pandas 的DataFrame 转化为numpy 使用values 就行了。
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y, type(inputs), type(inputs.values)

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]),
 pandas.core.frame.DataFrame,
 numpy.ndarray)